In [3]:
# EXTERNAL LIBRARIES
import numpy as np
import matplotlib.pyplot as plt
import torch
import yaml
from pathlib import Path
import os
import sys

# MY SCRIPTS
sys.path.append('functions/') # scripts folder
%load_ext autoreload
%autoreload 2 # Reload all modules (except those excluded by %aimport) every time before executing the Python code typed
import plots
import system as syst
import qammod as qmd
import papreduce as prd
import ccdf as ccdf
import file as f

# Detect if we are in Google Colaboratory
try:
    import google.colab
    IN_COLAB = True
    print('You are going to use GOOGLE COLAB')
except ImportError:
    IN_COLAB = False
    print('You are going to use LOCAL MACHINE')

# Add path for scripts and files directory
if IN_COLAB:
    google.colab.drive.mount("/content/drive")
    AUX_DATA_ROOT = Path("/content/drive/My Drive/PAPR reduction/Code")
    assert AUX_DATA_ROOT.is_dir(), "Have you forgot to 'Add a shortcut to Drive'?"
    sys.path.append(str(AUX_DATA_ROOT))
else:
    AUX_DATA_ROOT = ''
    
CFG_PATH = os.path.join(AUX_DATA_ROOT, 'config.yaml')
      
if torch.cuda.is_available():
    device=torch.device('cuda')
    print('You are going to use CUDA')
else:
    device=torch.device('cpu')
    print('You are going to use CPU')

You are going to use LOCAL MACHINE
You are going to use CUDA


In [4]:
# OPEN CONFIGURATION FILE
with open('config.yaml','r') as file:
    cfg = yaml.load(file, Loader=yaml.FullLoader)
    print('Config has been readed successfully')

Fs = cfg['Fs']
N_UE = cfg['N_UE']
P_UE_max_min = cfg['P_UE_max_min']
P_TX = cfg['P_TX']
N_RB = cfg['N_RB']
N_fft = cfg['N_fft']
N_zero = cfg['N_zero']
N_used = cfg['N_used']

EVM_QPSK = cfg['EVM_QPSK']
EVM_QAM16 = cfg['EVM_QAM16']
EVM_QAM64 = cfg['EVM_QAM64']
EVM_QAM256 = cfg['EVM_QAM256']
# EVM_QAM1024 = cfg['EVM_QAM1024']

M = cfg['M']
N_SC_RB = cfg['N_SC_RB']

# load complex and decimal modulation constellations
constellations = qmd.load_constellations(config=cfg,device=device,info=True)

# load Fourier matrix
S_sc = prd.gen_fourier_matrix(config=cfg,device=device)

# PAPR range for CCDF
PAPR_RANGE = torch.arange(0,12.25,step=0.25)

# zero shift to center spectrum
ZERO_SHIFT = N_zero//2

Config has been readed successfully
The following constelletions have been loaded: ['QPSK', 'QAM16', 'QAM64', 'QAM256']


In [5]:
# ADDITIONAL IMPORTS
from scipy.linalg import dft

In [17]:
np.set_printoptions(precision=6, suppress=False)
F = dft(1024)

torch.isclose(torch.tensor(F,dtype=torch.complex64),S_sc.cpu())

tensor([[ True,  True,  True,  ...,  True,  True,  True],
        [ True,  True,  True,  ...,  True,  True,  True],
        [ True,  True,  True,  ...,  True,  True,  True],
        ...,
        [ True,  True,  True,  ..., False, False, False],
        [ True,  True,  True,  ..., False, False, False],
        [ True,  True,  True,  ..., False, False, False]])

In [20]:
torch.sum(torch.tensor(F,dtype=torch.complex64)-S_sc.cpu().conj().T)

tensor(0.1592+0.0109j)